<a href="https://colab.research.google.com/github/arinamur/Classification/blob/main/CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_res = pd.read_excel('drive/MyDrive/new_case_dataset.xlsx')

In [4]:
spheres = ['Жилищно-коммунальное хозяйство', 'Здравоохранение', 'Благоустройство населённых пунктов (площади, парки, скверы, улицы, дворовые территории, кладбища)', 'Автомобильные дороги', 'Социальное обеспечение населения', 'Общественный транспорт (ОТ)', 'Образование', 'Неправомерная деятельность органов власти', 'Правопорядок', 'Экология', 'Трудовые отношения', 'Строительство, градостроительная деятельность']

In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
import string
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

import re
def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
mystem = Mystem() 

russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'здравствуйте', 'который', 'это', 'пожалуйста', 'спасибо', 'этот', 'наш', 'никто', 'свой'])

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [7]:
!pip install pymorphy2
!pip install catboost

     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 8.2 MB 17.1 MB/s 
     |████████████████████████████████| 76.1 MB 1.1 MB/s 


In [8]:
df_res

,sphere,category,theme,text,text_prep,text_stem,text_lemm
0,0,Многоквартирные дома,Нарушения в вопросах оплаты услуг ЖКХ,обращаются жильцы можем заверить что и других...,обращаются жильцы можем заверить других домов ...,обраща жильц мож завер друг дом наш микрорайон...,обращаться жилец мочь заверить дом микрорайон ...
1,0,Многоквартирные дома,Содержание МКД и придомовой территории *,мая происходит залитие подвала многоквартирно...,мая происходит залитие подвала многоквартирног...,ма происход залит подва многоквартирн дом фека...,май происходить залитие подвал многоквартирный...
2,0,"Государственный, муниципальный и ведомственный...","Обеспечение жильем ветеранов, инвалидов и семе...",вечер ребенка инвалида являюсь матерью одиноч...,вечер ребенка инвалида являюсь матерью одиночк...,вечер ребенк инвалид явля матер одиночк обрат ...,вечер ребёнок инвалид являться мать одиночка о...
3,0,Многоквартирные дома,Капитальный ремонт многоквартирного дома *,Здравствуйте нашего дома находится в полуразру...,нашего дома находится полуразрушенном состояни...,наш дом наход полуразрушен состоян управлен жи...,дом находиться полуразрушить состояние управле...
4,0,Многоквартирные дома,Нарушения в вопросах оплаты услуг ЖКХ,Здравствуйте зовут вопрос касается отопления и...,зовут вопрос касается отопления сумм которые н...,зовут вопрос каса отоплен сумм котор нам квита...,звать вопрос касаться отопление сумма квитанци...
...,...,...,...,...,...,...,...
7387,11,Строительный объект,Нарушение строительных норм *,день жители частного сектора домов номер прос...,день жители частного сектора домов номер проси...,ден жител частн сектор дом номер прос помощ ра...,день житель частное сектор дом номер просить п...
7388,11,Строительный объект,Неисполнение застройщиком договорных обязательств,день в активно ведётся строительство долгостр...,день активно ведётся строительство долгостроев...,ден активн ведет строительств долгостро везд з...,день активно вестись строительство долгострой ...
7389,11,Строительный объект,Неисполнение застройщиком договорных обязательств,Здравствуйте я ранее обращалась дома пришел от...,ранее обращалась дома пришел ответ следующего ...,ран обраща дом пришел ответ след содержан обра...,ранее обращаться дом прийти ответ следующий со...
7390,11,Строительный объект,Неисполнение застройщиком договорных обязательств,приходом должность главы региона началось реш...,приходом должность главы региона началось реше...,приход должност глав регион нача решен проблем...,приход должность глава регион начаться решение...


In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [11]:
from sklearn.model_selection import cross_validate, StratifiedKFold

In [12]:
from catboost import CatBoostClassifier
cbc = CatBoostClassifier(random_state=42, n_estimators=20, max_depth=5)

In [13]:
X, y = [], []
for i, row in df_res.iterrows():
  th = remove_multiple_spaces(remove_numbers(remove_punctuation(str(row.theme).strip().lower())))
  X.append(th + ' ' + row.text_prep)
  y.append(row.sphere)
X, y = np.array(X), np.array(y)

In [14]:
model = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', cbc),
               ])

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_validate(estimator=model, X=X, y=y, 
                         cv=skf, scoring=['f1_weighted', 'accuracy'], n_jobs=-1)
for i in scores.keys():
  print('{}\n scores = {} \nmean score = {:.5f} +/- {:.5f}'.format(i, scores[i], scores[i].mean(), scores[i].std()))

fit_time
 scores = [56.04514885 56.45455813 45.24751544 45.11590743 23.73870993] 
mean score = 45.32037 +/- 11.87271
score_time
 scores = [0.34331942 0.24377942 0.25076032 0.23148108 0.15661359] 
mean score = 0.24519 +/- 0.05949
test_f1_weighted
 scores = [0.92332055 0.88638578 0.88768235 0.90342594 0.89108767] 
mean score = 0.89838 +/- 0.01385
test_accuracy
 scores = [0.92359703 0.88911427 0.88903924 0.9052774  0.89242219] 
mean score = 0.89989 +/- 0.01327
